In [1]:
# if (!require("BiocManager", quietly = TRUE))
#     install.packages("BiocManager")
# BiocManager::install(version = "3.10")
# BiocManager::install("randomForest")

In [2]:
setwd('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Input')
Output = ('/Users/alexis/University of North Carolina at Chapel Hill/Rager, Julia - CEMALB_DataAnalysisPM/Projects/P1002. Ecig Metabolomics NELF/P1002.3. Analyses/P1002.3.2. Smoking Status Prediction/Output')
cur_date = "030922"

library(tidyverse)
library(caret)
library(e1071)
library(Hmisc)
library(glmnet)
library(pROC)
library(randomForest)

#reading in file
molecular_feature_df_w_Ecig_40 = read_csv("20220217_mf_qrilc_w40.csv") %>%
     rename(Smoking_Status = sample)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.3     ✔ purrr   0.3.4
✔ tibble  3.0.6     ✔ dplyr   1.0.4
✔ tidyr   1.1.2     ✔ stringr 1.4.0
✔ readr   1.4.0     ✔ forcats 0.5.1

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The following object is masked from ‘package:caret’:

    cluster


Loading required package: Formula


Attaching package: ‘Hmisc’


The following object is masked from ‘package:e1071’:

    impute


The following objects are masked from ‘package:dplyr’:

    src, summarize


The following objects are masked from ‘package:base’:

    format.pval, units


Loading required package: Matrix


Attaching package: ‘Mat

In [3]:
head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
NS_070,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
NS_110,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
NS_143,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
NS_152,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
NS_187,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
NS_196,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


In [4]:
#first making smoking status column binary denoted by 0 = non smoker and 1 = ecig user
molecular_feature_df_w_Ecig_40 = molecular_feature_df_w_Ecig_40 %>%
    #grepl is synonymous with "contains"
    #necessary to put it into factor and relevel, so algorithm understands which data is considered baseline
    mutate(Smoking_Status = relevel(factor(ifelse(grepl("Ecig", Smoking_Status), 1, 0)), ref = "0"))

mf_df_w_Ecig_40_wo_nicotine = molecular_feature_df_w_Ecig_40 %>%
    #removing nicotine and cotinine
    select(-c(MF_97, MF_5))

head(molecular_feature_df_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_62853,MF_63431,MF_63701,MF_64025,MF_64936,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06974,19.10260,17.17255,16.92465,17.09794,19.06203,19.13556,16.93182,17.55702,17.05593
0,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.02560,17.82667,19.37426,16.23450,19.06421,18.06379,19.39103,16.72112,19.27343,19.30881
0,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.14095,19.02411,19.56175,17.17120,16.97302,19.49121,17.09470,15.56214,19.55038,19.54912
0,21.60361,23.22761,21.99010,16.22547,17.59020,21.53908,21.14531,22.07857,21.97502,⋯,19.28751,19.54886,16.78848,16.46750,19.47635,19.44477,16.99617,16.09531,19.28810,19.32788
0,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,16.70042,17.85969,18.97406,18.18813,18.50336,19.19085,19.41757,16.88993,19.35678,18.95550
0,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.86999,15.82637,19.46467,16.97149,17.46699,16.32598,15.62938,17.37207,15.84334,17.82000


The goal of this analysis is to use molecular features [continuous] to predict smoking staus (dichotomous). This will be done using Support Vector Machine (SVM) based on a logistic regression model for classification and Random Forest.

# Random Forest

- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [5]:
rf_classification = function(dataset, outcome, pred_outcome){
    #setting for reproducibility
    set.seed(12)
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV

    metrics = data.frame()
    variable_importance_df = data.frame()
    
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        ntree_values = c(50, 250, 500) #number of trees (not sure if we wanted something different)
        p = dim(dataset)[2] - 1 #number of variables in dataset
        mtry_values = c(p/2, sqrt(p), p)

        #will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        #finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = data_train,
        #need to go back and figure out how the number of trees/ variables make a difference even though
        #the errors are the same!!!!
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)], 
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        #predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")


        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        #extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "MF")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
    }
    
    #taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(MF) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        #sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    #return training set, matrix and variable importance values
    return(list(data_train, metrics, variable_importance_df))

}

#calling fn
rf_values_w_Ecig_40 = rf_classification(molecular_feature_df_w_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
rf_values_w_Ecig_40_wo_nicotine = rf_classification(mf_df_w_Ecig_40_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [8]:
#viewing results
rf_confusion_matrix = data.frame(Model = c(rep(c("Original RF"), times = 2)), Removed.Variables = c("None", "Nicotine & Cotinine"),
                                 rbind(rf_values_w_Ecig_40[[2]], rf_values_w_Ecig_40_wo_nicotine[[2]]))
rf_confusion_matrix

#viewing most significant features
head(rf_values_w_Ecig_40[[3]])
head(rf_values_w_Ecig_40_wo_nicotine[[3]])

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,None,0.9047619,0.8428571,0.9666667,0.9600000,0.9047619
Original RF,Nicotine & Cotinine,0.6185714,0.6571429,0.5800000,0.6883333,0.6185714


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,7.0542406
MF_5,4.6837830
MF_292,0.2770581
MF_3345,0.2480095
MF_1713,0.1876070
MF_4803,0.1654220


MF,MeanDecreaseGini
<chr>,<dbl>
MF_292,0.8834351
MF_7221,0.5365767
MF_3345,0.4643966
MF_5547,0.4396756
MF_1324,0.3772763
MF_1713,0.3212409


In [10]:
#exporting mean gini decrease values
# write.csv(rf_values_w_Ecig_40[[3]], paste0(Output,"/", cur_date, "_Feature_Ranking.csv"), row.names = FALSE)
# write.csv(rf_values_wo_Ecig_40[[3]], paste0(Output,"/", cur_date, "_Feature_Ranking_wo_Ecig_40.csv"), row.names = FALSE)
# write.csv(rf_values_w_Ecig_40_wo_nicotine[[3]], paste0(Output,"/", cur_date, "_Feature_Ranking_wo_Nictotine.csv"), row.names = FALSE)
# write.csv(rf_values_wo_Ecig_40_wo_nicotine[[3]], paste0(Output,"/", cur_date, "_Feature_Ranking_wo_Ecig_40_wo_Nictotine.csv"), row.names = FALSE)

Rerunning RF model with noise variables to determine, which predictors are significant above the background noise. 

In [9]:
train_vars_noise_w_Ecig_40 = rf_values_w_Ecig_40[[1]]
train_vars_noise_w_Ecig_40_wo_nicotine = rf_values_w_Ecig_40_wo_nicotine[[1]]

noise_df = function(train_vars_noise){
    set.seed(7)
    # Add random noise predictors as an additional method to evaluate model performance
    # Adding a column that contains randomly shuffled values from one of the molecules; sampling with replacement
    train_vars_noise$noise1 = sample(train_vars_noise[[colnames(train_vars_noise[3])]], replace = TRUE) 
    train_vars_noise$noise2 = sample(train_vars_noise[[colnames(train_vars_noise[4])]], replace = TRUE)
    train_vars_noise$noise3 = sample(train_vars_noise[[colnames(train_vars_noise[5])]], replace = TRUE)
    train_vars_noise$noise4 = sample(train_vars_noise[[colnames(train_vars_noise[6])]], replace = TRUE)
    train_vars_noise$noise5 = sample(train_vars_noise[[colnames(train_vars_noise[7])]], replace = TRUE)
    
    return(train_vars_noise)
}

#calling fn
noise_training_rf_dataset_w_Ecig_40 = noise_df(train_vars_noise_w_Ecig_40)
noise_training_rf_dataset_w_Ecig_40_wo_nicotine = noise_df(train_vars_noise_w_Ecig_40_wo_nicotine)
head(noise_training_rf_dataset_w_Ecig_40)

Smoking_Status,MF_2,MF_3,MF_4,MF_5,MF_6,MF_7,MF_11,MF_12,MF_13,⋯,MF_65532,MF_68286,MF_73340,MF_77165,MF_78976,noise1,noise2,noise3,noise4,noise5
<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
0,19.86186,17.86943,19.21394,18.02809,17.46636,18.34174,18.41799,19.31361,19.46777,⋯,19.06203,19.13556,16.93182,17.55702,17.05593,22.95603,21.55396,19.64002,19.91744,22.63653
0,21.08622,22.58769,21.05378,17.93584,17.33098,20.41479,20.37485,21.12876,21.01488,⋯,18.06379,19.39103,16.72112,19.27343,19.30881,23.00581,22.91377,15.77071,17.88844,23.46244
0,13.12099,14.44659,13.46827,16.51009,14.93820,15.66299,15.72731,13.78698,15.18975,⋯,19.49121,17.09470,15.56214,19.55038,19.54912,22.86175,22.54138,20.63665,17.31698,22.06629
0,19.43885,17.93876,19.68977,16.73360,17.88844,17.81758,17.64979,19.64269,19.32233,⋯,19.19085,19.41757,16.88993,19.35678,18.95550,24.32335,22.14102,16.08028,17.71179,23.15163
0,22.53459,24.36662,22.77595,16.69956,16.38411,22.55912,17.30008,22.80146,22.09338,⋯,16.32598,15.62938,17.37207,15.84334,17.82000,22.73364,20.20755,21.28516,26.11076,22.61793
0,21.50216,23.34553,21.53633,17.93372,24.26751,21.71412,21.32122,21.66214,21.71152,⋯,16.71579,18.52501,17.61326,17.74147,16.76017,22.58769,19.91826,16.08028,15.36472,22.43186


In [10]:
#calling rf fn
noise_rf_values_w_Ecig_40 = rf_classification(noise_training_rf_dataset_w_Ecig_40, "Smoking_Status", "pred_Smoking_Status")
noise_rf_values_w_Ecig_40_wo_nicotine = rf_classification(noise_training_rf_dataset_w_Ecig_40_wo_nicotine, "Smoking_Status", "pred_Smoking_Status")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [11]:
#viewing results
noise_rf_confusion_matrix = data.frame(Model = c(rep(c("RF w/ Noise"), times = 2)), Removed.Variables = c("None", "Nicotine & Cotinine"), 
                                       rbind(noise_rf_values_w_Ecig_40[[2]], noise_rf_values_w_Ecig_40_wo_nicotine[[2]]))
noise_rf_confusion_matrix

#filtering for most significant features
head(noise_rf_values_w_Ecig_40[[3]])
head(noise_rf_values_w_Ecig_40_wo_nicotine[[3]]) 

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF w/ Noise,None,0.8783333,0.9266667,0.83,0.8809524,0.8783333
RF w/ Noise,Nicotine & Cotinine,0.5600000,0.5800000,0.54,0.6600000,0.5600000


MF,MeanDecreaseGini
<chr>,<dbl>
MF_97,2.6118845
MF_5,1.8559144
MF_292,0.7185532
MF_7221,0.3767431
MF_29367,0.2427987
MF_6955,0.1876891


MF,MeanDecreaseGini
<chr>,<dbl>
MF_292,0.5308698
MF_3345,0.3398274
MF_7221,0.2814058
MF_6955,0.2727071
MF_1522,0.2459021
MF_34412,0.1909755


# Support Vector Machine
- supervised learning models that can predict continuous (regression) or grouped/dichotomous (classification) outcomes
- predictors by projecting them onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [12]:
#model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    #setting seed for reproducibility
    set.seed(12)
    
    #splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        data_test = dataset[dataset_index[[i]],]

        #now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        #choosing best model
        best_svm <- svr_tune$best.model

        #predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        #calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        #extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        #adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  #taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(metrics)
}

In [13]:
#calling fn
svm_w_Ecig_40 = svm_classification(molecular_feature_df_w_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
svm_w_Ecig_40_wo_nicotine = svm_classification(mf_df_w_Ecig_40_wo_nicotine, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [15]:
#creating one combined df
svm_final_df = data.frame(rbind(svm_w_Ecig_40, svm_w_Ecig_40_wo_nicotine))

svm_final_df = data.frame(Model = c(rep(c("Original SVM"), times = 2)), Removed.Variables = c("None", "Nicotine & Cotinine"), svm_final_df)
svm_final_df

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original SVM,None,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Original SVM,Nicotine & Cotinine,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286


# RF Variable Selection w/ SVM

Calculating the average noise rank to determine, which predictors ranked higher than the highest noise rank. Then filtering the original datasets for those predictors and rerunning SVM.

In [50]:
#calculating avg noise rank out of all variables
noise_importance_values_w_Ecig_40 = noise_rf_values_w_Ecig_40[[3]] %>%
    column_to_rownames(var = "MF")
noise_importance_values_w_Ecig_40_wo_nicotine = noise_rf_values_w_Ecig_40_wo_nicotine[[3]] %>%
    column_to_rownames(var = "MF")

In [51]:
highest_noise_rank = function(noise_importance_values){
    #this gives us values between 0 and 1
    #the closer the number gets to 1 the lower it ranks, so the number closest to 1 is the max rank
    noise1rank = which(rownames(noise_importance_values) == "noise1")/nrow(noise_importance_values)
    noise2rank = which(rownames(noise_importance_values) == "noise2")/nrow(noise_importance_values)
    noise3rank = which(rownames(noise_importance_values) == "noise3")/nrow(noise_importance_values)
    noise4rank = which(rownames(noise_importance_values) == "noise4")/nrow(noise_importance_values)
    noise5rank = which(rownames(noise_importance_values) == "noise5")/nrow(noise_importance_values)

    #figuring out which is the highest mean decrease gini or background noise importance value
    highest_noise_rank = min(noise1rank, noise2rank, noise3rank, noise4rank, noise5rank)
    
    return(highest_noise_rank)
}

#calling fn
highest_noise_rank_w_Ecig_40 = highest_noise_rank(noise_importance_values_w_Ecig_40)
highest_noise_rank_w_Ecig_40_wo_nicotine = highest_noise_rank(noise_importance_values_w_Ecig_40_wo_nicotine)
highest_noise_rank_w_Ecig_40
highest_noise_rank_w_Ecig_40_wo_nicotine

[1] 0.2073768

[1] 0.02167742

In [52]:
last_significant_rank = function(noise_importance_values, highest_noise_rank){
    for (i in 1:length(noise_importance_values$MeanDecreaseGini)){
    #if the row number (index)/# of variables is less than the highest noise rank the loop will continue
        if (i/length(noise_importance_values$MeanDecreaseGini) > highest_noise_rank){
            last_significant_var = i - 1
            break
        }
    }    
    return(last_significant_var)
}

#calling fn
last_significant_rank_w_Ecig_40 = last_significant_rank(noise_importance_values_w_Ecig_40, highest_noise_rank_w_Ecig_40)
last_significant_rank_w_Ecig_40_wo_nicotine = last_significant_rank(noise_importance_values_w_Ecig_40_wo_nicotine, highest_noise_rank_w_Ecig_40_wo_nicotine)
last_significant_rank_w_Ecig_40
last_significant_rank_w_Ecig_40_wo_nicotine

[1] 804

[1] 84

The dataset containing all the features contains 804 MFs above background, while the dataset without nicotine and cotinine has 84. This means that the exclusion of these 2 features forces the model to find erroneous connections between the noise variables and the outcome and that these 2 features are driving much of the model's predictive ability. 

In [53]:
#now getting those significant MFs
significant_MFs_w_Ecig_40 = rownames(noise_importance_values_w_Ecig_40)[1:last_significant_rank_w_Ecig_40]
significant_MFs_w_Ecig_40_wo_nicotine = rownames(noise_importance_values_w_Ecig_40_wo_nicotine)[1:last_significant_rank_w_Ecig_40_wo_nicotine]

#now subsetting the original dfs
rf_subset_MF_df_w_Ecig_40 = cbind(Smoking_Status = molecular_feature_df_w_Ecig_40$Smoking_Status, molecular_feature_df_w_Ecig_40[,colnames(molecular_feature_df_w_Ecig_40) %in% significant_MFs_w_Ecig_40])
rf_subset_MF_df_w_Ecig_40_wo_nicotine = cbind(Smoking_Status = mf_df_w_Ecig_40_wo_nicotine$Smoking_Status, mf_df_w_Ecig_40_wo_nicotine[,colnames(mf_df_w_Ecig_40_wo_nicotine) %in% significant_MFs_w_Ecig_40_wo_nicotine])
head(rf_subset_MF_df_w_Ecig_40)

,Smoking_Status,MF_5,MF_13,MF_17,MF_35,MF_37,MF_39,MF_46,MF_53,MF_54,⋯,MF_51718,MF_53468,MF_53619,MF_55392,MF_59362,MF_61014,MF_61833,MF_62137,MF_64936,MF_78976
,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,⋯,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
1,0,18.02809,19.46777,20.11458,17.82672,18.26643,19.00952,19.65691,18.63728,19.13422,⋯,19.11038,19.14811,18.26770,19.47902,19.14311,16.40603,16.89474,17.47760,17.09794,17.05593
2,0,17.93584,21.01488,19.82979,16.81001,19.51374,17.87427,21.82161,22.64031,21.19316,⋯,17.62661,18.07494,19.82107,18.94722,18.14936,19.55914,20.04172,16.35027,19.06421,19.30881
3,0,16.51009,15.18975,18.65454,19.16474,21.11230,16.80061,19.56197,16.47404,13.40680,⋯,19.19945,17.35692,18.97545,17.44090,21.24468,19.35598,20.66664,15.64893,16.97302,19.54912
4,0,16.22547,21.97502,18.35221,18.66030,21.16445,17.69103,19.20528,22.71431,22.19120,⋯,20.02821,19.98383,19.31315,16.84370,15.54892,19.12636,15.82487,17.78108,19.47635,19.32788
5,0,16.73360,19.32233,20.23552,17.94368,19.56851,17.53120,18.81662,17.17345,19.73012,⋯,17.46290,19.37300,19.69692,17.48581,19.98009,17.56106,17.89533,17.97428,18.50336,18.95550
6,0,16.69956,22.09338,20.72501,16.55946,24.81633,18.90936,20.70904,24.92124,23.10405,⋯,15.50206,18.54867,18.53569,16.89375,16.54808,18.71601,19.58488,16.06782,17.46699,17.82000


In [55]:
#getting dataframe ready to export variable importance ranks from above and whether those features passed the 
# background filter
significant_MFs_df = data.frame(MF = significant_MFs_w_Ecig_40, Passed_Filter = c(rep(c("Yes"), 
                                                                    times = length(significant_MFs_w_Ecig_40))))
significant_MFs_df_wo_nicotine = data.frame(MF = significant_MFs_w_Ecig_40_wo_nicotine, Passed_Filter = c(rep(c("Yes"), 
                                                                    times = length(significant_MFs_w_Ecig_40_wo_nicotine))))

significant_MFs_df = full_join(significant_MFs_df, rf_values_w_Ecig_40[[3]])
significant_MFs_df_wo_nictoine = full_join(significant_MFs_df_wo_nicotine, rf_values_w_Ecig_40_wo_nicotine[[3]])

head(significant_MFs_df_wo_nictoine)

Joining, by = "MF"

Joining, by = "MF"



,MF,Passed_Filter,MeanDecreaseGini
,<chr>,<fct>,<dbl>
1,MF_292,Yes,0.88343510
2,MF_3345,Yes,0.46439663
3,MF_7221,Yes,0.53657673
4,MF_6955,Yes,0.04491567
5,MF_1522,Yes,0.08419519
6,MF_34412,Yes,0.15137875


In [56]:
#exporting those features that passed the background filter
write.csv(significant_MFs_df, paste0(Output,"/", cur_date, "_Significant_MFs.csv"), row.names = FALSE)
write.csv(significant_MFs_df_wo_nictoine, paste0(Output,"/", cur_date, "_Significant_MFs_wo_nicotine.csv"), row.names = FALSE)

In [57]:
#calling fn
subset_svm_w_Ecig_40 = svm_classification(rf_subset_MF_df_w_Ecig_40, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))
subset_svm_w_Ecig_40_wo_nicotine = svm_classification(rf_subset_MF_df_w_Ecig_40_wo_nicotine, "linear", "Smoking_Status", "pred_Smoking_Status",
                                            0.035, 1:5, seq(0,1,0.2))

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [58]:
#creating one combined df
subset_svm_final_df = data.frame(rbind(subset_svm_w_Ecig_40, subset_svm_w_Ecig_40_wo_nicotine))

subset_svm_final_df = data.frame(Model = c(rep(c("SVM w/ RF Var Selection"), times = 2)), Removed.Variables = c("None", "Nicotine & Cotinine"), subset_svm_final_df)
subset_svm_final_df

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM w/ RF Var Selection,None,0.8490476,0.8714286,0.8266667,0.8600000,0.8490476
SVM w/ RF Var Selection,Nicotine & Cotinine,0.7695238,0.8190476,0.7200000,0.7880952,0.7695238


In [59]:
#all results
final_df = rbind(rf_confusion_matrix, svm_final_df, subset_svm_final_df)
final_df

Model,Removed.Variables,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<fct>,<fct>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Original RF,None,0.9047619,0.8428571,0.9666667,0.9600000,0.9047619
Original RF,Nicotine & Cotinine,0.6185714,0.6571429,0.5800000,0.6883333,0.6185714
Original SVM,None,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
Original SVM,Nicotine & Cotinine,0.7614286,0.8095238,0.7133333,0.8044444,0.7614286
SVM w/ RF Var Selection,None,0.8490476,0.8714286,0.8266667,0.8600000,0.8490476
SVM w/ RF Var Selection,Nicotine & Cotinine,0.7695238,0.8190476,0.7200000,0.7880952,0.7695238


In [60]:
#exporting results
write.csv(final_df, paste0(Output,"/", cur_date, "_Smoking_Status_Prediction_Results.csv"), row.names = FALSE)